In [1]:
import tensorflow

In [2]:
pip install optuna

  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/386.6 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/386.6 kB 991.0 kB/s eta 0:00:01
   --------- ------------------------------ 92.2/386.6 kB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 184.3/386.6 kB 1.6 MB/s eta 0:00:01
   ------------------- -------------------- 184.3/386.6 kB 1.6 MB/s eta 0:00:01
   ------------------- -------------------- 184.3/386.6 kB 1.6 MB/s eta 0:00:01
   ------------------------------ --------- 297.0/386.6 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 368.6/386.6 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 386.6/386.6 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/231.9 kB ? eta -:--:--
   ---------- ----------------------------- 61.4/231.9 kB 3.4 MB/s eta 0:00:01
   --------------- ------------------------ 92.2/231.9 kB 1.7 MB/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires tensorboard~=2.19.0, but you have tensorboard 2.18.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [9]:
import optuna

In [5]:
# requirements

import pandas as pd
from sklearn.preprocessing import LabelEncoder

import tensorflow
from tensorflow import keras

from keras.metrics import Precision,Recall,F1Score
df = pd.read_csv(r"C:\Users\prade\OneDrive\Desktop\optuna\data.csv")
df.drop(columns=["Timestamp","Email Address"],axis=1,inplace=True)
for col in df.columns:
    le = LabelEncoder()
    le = le.fit(df[col])
    df[col] = le.fit_transform(df[col])

from sklearn.cluster import KMeans

obj = KMeans(n_clusters=2,n_init=5).fit(df)
clusters = obj.labels_

df["labels"] = clusters

x = df.drop(columns="labels",axis=1)
y = df["labels"]

model = keras.Sequential()
model.add(keras.layers.Input(shape=(10,)))
model.add(keras.layers.Dense(8,
                             kernel_initializer=keras.initializers.GlorotNormal(seed=42),
                             ))
model.add(keras.layers.PReLU())
model.add(keras.layers.Dense(3,
                             kernel_initializer=keras.initializers.GlorotNormal(seed=42),
                            kernel_regularizer = keras.regularizers.L2()))
model.add(keras.layers.PReLU())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation="sigmoid",
                            kernel_initializer=keras.initializers.HeNormal(seed=42)))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy",Precision(),Recall()])
model.fit(x,y,validation_split=0.2,batch_size=8,epochs=30)


Epoch 1/30


c:\Users\prade\anaconda3\envs\giri\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.5422 - loss: 0.8218 - precision_1: 0.6028 - recall_1: 0.6342 - val_accuracy: 0.6667 - val_loss: 0.7217 - val_precision_1: 0.6667 - val_recall_1: 1.0000
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7238 - loss: 0.6056 - precision_1: 0.7408 - recall_1: 0.8072 - val_accuracy: 0.6000 - val_loss: 0.7178 - val_precision_1: 0.6429 - val_recall_1: 0.9000
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6626 - loss: 0.5825 - precision_1: 0.7003 - recall_1: 0.6857 - val_accuracy: 0.5667 - val_loss: 0.7131 - val_precision_1: 0.6296 - val_recall_1: 0.8500
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7178 - loss: 0.5467 - precision_1: 0.7714 - recall_1: 0.7416 - val_accuracy: 0.6000 - val_loss: 0.7069 - val_precision_1: 0.6538 - val_recall_1: 0.8500
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7473 - loss: 0.4851 - precision_1: 0.7331 - recall_1: 0.8659 - val_accuracy: 0.766

In [6]:
import optuna

In [7]:
from keras import initializers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, PReLU, Input, Dropout, BatchNormalization, Activation
from keras.metrics import Precision, Recall



def objective(trial):
  model = Sequential()
  model.add(Input(shape=(10,)))
  num_layers = trial.suggest_int("num_layers", 1, 4)
  for i in range(num_layers):
        units = trial.suggest_int(f"units_l{i}", 4, 16)
        init_choice = trial.suggest_categorical(f"initializer_l{i}", ["glorot_normal", "he_normal", "glorot_uniform", "he_uniform"])
        reg_choice = trial.suggest_categorical(f"regularizer_l{i}", ["l1", "l2"])
        use_prelu = trial.suggest_categorical(f"use_prelu_l{i}", [True, False])

        initializer = getattr(initializers, init_choice)()
        regularizer = getattr(regularizers, reg_choice)(0.01)

        model.add(Dense(units=units, kernel_initializer=initializer, kernel_regularizer=regularizer))

        if use_prelu:
            model.add(PReLU())
            model.add(BatchNormalization())
            dropout_rate = trial.suggest_float(f"dropout_l{i}", 0.0, 0.5)
            model.add(Dropout(dropout_rate))
        else:
            activation = trial.suggest_categorical(f"activation_l{i}", ["relu", "sigmoid", "tanh"])
            model.add(Activation(activation))

  model.add(Dense(1, activation="sigmoid"))
  model.compile(optimizer=trial.suggest_categorical("optimizer", ["adam", "sgd", "adagrad"]),loss="binary_crossentropy",metrics=["accuracy", Precision(), Recall()])
  
  history = model.fit(x,y,epochs=30,batch_size=8,validation_split=0.2)
  return history.history['val_loss'][-1]




In [8]:
space = {
    "num_layers": [2],
    
    "units_l0": [4,16],
    "initializer_l0": ["glorot_normal"],
    "regularizer_l0": ["l2"],
    "dropout_l0": [0.2],
    "use_prelu_l0": [True,False],
    "activation_l0": ["relu","tanh","sigmoid"],
    
    "units_l1": [4,16],
    "initializer_l1": ["glorot_normal"],
    "regularizer_l1": ["l2"],
    "dropout_l1": [0.2],
    "use_prelu_l1": [True,False],
    "activation_l1": ["relu","tanh","sigmoid"],
    
    "optimizer": ["adam","sgd","adagrad"]
}


In [10]:
study = optuna.create_study(direction = "minimize", sampler = optuna.samplers.GridSampler(space))
study.optimize(objective, n_trials=30)

[I 2025-05-19 08:58:25,352] A new study created in memory with name: no-name-817c1906-0f40-48ea-ab88-1b8200853a45


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3869 - loss: 1.5201 - precision_12: 0.5084 - recall_12: 0.4004 - val_accuracy: 0.5000 - val_loss: 1.1769 - val_precision_12: 0.8571 - val_recall_12: 0.3000
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5004 - loss: 1.3109 - precision_12: 0.5848 - recall_12: 0.5878 - val_accuracy: 0.5333 - val_loss: 1.0930 - val_precision_12: 0.8750 - val_recall_12: 0.3500
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4914 - loss: 1.3174 - precision_12: 0.5872 - recall_12: 0.5902 - val_accuracy: 0.5333 - val_loss: 1.0172 - val_precision_12: 0.8000 - val_recall_12: 0.4000
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6366 - loss: 1.0290 - precision_12: 0.7477 - recall_12: 0.6138 - val_accuracy: 0.5333 - val_loss: 0.9543 - val_precision_12: 0.8000 - val_recall_12: 0.4000
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6016 - loss: 0.9546 - precision_12: 0.6238 - recall_12:

[I 2025-05-19 08:58:29,733] Trial 0 finished with value: 0.38021934032440186 and parameters: {'num_layers': 2, 'units_l0': 16, 'initializer_l0': 'glorot_normal', 'regularizer_l0': 'l2', 'use_prelu_l0': False, 'activation_l0': 'tanh', 'units_l1': 4, 'initializer_l1': 'glorot_normal', 'regularizer_l1': 'l2', 'use_prelu_l1': True, 'dropout_l1': 0.2, 'optimizer': 'adam'}. Best is trial 0 with value: 0.38021934032440186.


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.2933 - loss: 1.5304 - precision_13: 0.3696 - recall_13: 0.3189 - val_accuracy: 0.3333 - val_loss: 1.0998 - val_precision_13: 0.0000e+00 - val_recall_13: 0.0000e+00
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3573 - loss: 1.3637 - precision_13: 0.3779 - recall_13: 0.3419 - val_accuracy: 0.3333 - val_loss: 1.0431 - val_precision_13: 0.0000e+00 - val_recall_13: 0.0000e+00
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4179 - loss: 1.0496 - precision_13: 0.5136 - recall_13: 0.4347 - val_accuracy: 0.3333 - val_loss: 0.9955 - val_precision_13: 0.0000e+00 - val_recall_13: 0.0000e+00
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4754 - loss: 1.0031 - precision_13: 0.5821 - recall_13: 0.5678 - val_accuracy: 0.3333 - val_loss: 0.9527 - val_precision_13: 0.0000e+00 - val_recall_13: 0.0000e+00
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5893 - loss: 0.8785 - p

[I 2025-05-19 08:58:34,148] Trial 1 finished with value: 0.37116214632987976 and parameters: {'num_layers': 2, 'units_l0': 4, 'initializer_l0': 'glorot_normal', 'regularizer_l0': 'l2', 'use_prelu_l0': False, 'activation_l0': 'sigmoid', 'units_l1': 16, 'initializer_l1': 'glorot_normal', 'regularizer_l1': 'l2', 'use_prelu_l1': True, 'dropout_l1': 0.2, 'optimizer': 'adam'}. Best is trial 1 with value: 0.37116214632987976.


...

[I 2025-05-19 09:02:48,429] Trial 27 finished with value: 0.2471729964017868 and parameters: {'num_layers': 2, 'units_l0': 4, 'initializer_l0': 'glorot_normal', 'regularizer_l0': 'l2', 'use_prelu_l0': True, 'dropout_l0': 0.2, 'units_l1': 4, 'initializer_l1': 'glorot_normal', 'regularizer_l1': 'l2', 'use_prelu_l1': True, 'dropout_l1': 0.2, 'optimizer': 'sgd'}. Best is trial 26 with value: 0.23035600781440735.


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.4583 - loss: 1.1406 - precision_40: 0.5579 - recall_40: 0.4155 - val_accuracy: 0.6000 - val_loss: 0.8739 - val_precision_40: 0.6667 - val_recall_40: 0.8000
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5640 - loss: 0.9521 - precision_40: 0.6203 - recall_40: 0.5637 - val_accuracy: 0.6333 - val_loss: 0.8370 - val_precision_40: 0.6957 - val_recall_40: 0.8000
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6438 - loss: 0.8295 - precision_40: 0.7994 - recall_40: 0.5640 - val_accuracy: 0.6000 - val_loss: 0.8272 - val_precision_40: 0.6818 - val_recall_40: 0.7500
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6416 - loss: 0.8235 - precision_40: 0.8142 - recall_40: 0.5510 - val_accuracy: 0.7000 - val_loss: 0.8041 - val_precision_40: 0.7895 - val_recall_40: 0.7500
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6447 - loss: 0.8091 - precision_40: 0.7853 - recall

[I 2025-05-19 09:02:59,382] Trial 28 finished with value: 0.38065147399902344 and parameters: {'num_layers': 2, 'units_l0': 16, 'initializer_l0': 'glorot_normal', 'regularizer_l0': 'l2', 'use_prelu_l0': True, 'dropout_l0': 0.2, 'units_l1': 4, 'initializer_l1': 'glorot_normal', 'regularizer_l1': 'l2', 'use_prelu_l1': True, 'dropout_l1': 0.2, 'optimizer': 'sgd'}. Best is trial 26 with value: 0.23035600781440735.


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.6072 - loss: 1.0018 - precision_41: 0.6767 - recall_41: 0.6410 - val_accuracy: 0.3333 - val_loss: 1.1548 - val_precision_41: 0.0000e+00 - val_recall_41: 0.0000e+00
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6306 - loss: 0.9797 - precision_41: 0.7676 - recall_41: 0.6097 - val_accuracy: 0.3333 - val_loss: 1.0669 - val_precision_41: 0.0000e+00 - val_recall_41: 0.0000e+00
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7502 - loss: 0.8510 - precision_41: 0.8752 - recall_41: 0.6466 - val_accuracy: 0.4000 - val_loss: 0.9941 - val_precision_41: 1.0000 - val_recall_41: 0.1000
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7992 - loss: 0.7784 - precision_41: 0.8756 - recall_41: 0.7740 - val_accuracy: 0.5667 - val_loss: 0.9250 - val_precision_41: 1.0000 - val_recall_41: 0.3500
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8071 - loss: 0.6834 - precision_41:

[I 2025-05-19 09:03:11,377] Trial 29 finished with value: 0.38790374994277954 and parameters: {'num_layers': 2, 'units_l0': 16, 'initializer_l0': 'glorot_normal', 'regularizer_l0': 'l2', 'use_prelu_l0': True, 'dropout_l0': 0.2, 'units_l1': 16, 'initializer_l1': 'glorot_normal', 'regularizer_l1': 'l2', 'use_prelu_l1': True, 'dropout_l1': 0.2, 'optimizer': 'adam'}. Best is trial 26 with value: 0.23035600781440735.


In [ ]:
study.trials_dataframe()